# Compare the similarity by numpy array between news/content and Ads.


Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 3.0) kernel, suggest 4vCPU+ 16GiB. 

#### In this notebook, we will compare the contents embedding with ads embedding, and find out the most relevant Ads with the contents. 
#### There are four steps -
1. Compare the items similarity
2. Re-ranking by the score, from higher to lower
3. Check the results
4. Save the results.


In [24]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
import pandas as pd
import json

# Load the JSON data from content_vectors.json
with open('content_vectors.json', 'r') as file:
    data = json.load(file)

# Create a DataFrame from the loaded JSON data
df1 = pd.DataFrame(data)

# Display the DataFrame
df1.info ()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16787 entries, 0 to 16786
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   newsdesk    16787 non-null  object
 1   section     16787 non-null  object
 2   subsection  16787 non-null  object
 3   material    16787 non-null  object
 4   headline    16787 non-null  object
 5   abstract    16787 non-null  object
 6   keywords    16787 non-null  object
 7   word_count  16787 non-null  int64 
 8   pub_date    16787 non-null  object
 9   n_comments  16787 non-null  int64 
 10  uniqueID    16787 non-null  object
 11  np_array    16787 non-null  object
 12  embedding   16787 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.7+ MB


In [59]:
df1.head ()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,n_comments,uniqueID,np_array,embedding
0,Editorial,Opinion,Unknown,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01 00:18:54+00:00,186,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,"[-1.0, -0.084472656, -0.56640625, -0.19824219,...","[-1.0, -0.084472656, -0.56640625, -0.19824219,..."
1,Games,Crosswords & Games,Unknown,News,‘It’s Green and Slimy’,Christina Iverson and Jeff Chen ring in the Ne...,['Crossword Puzzles'],931,2020-01-01 03:00:10+00:00,257,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,"[-0.69921875, 0.66015625, 0.33789062, -0.46289...","[-0.69921875, 0.66015625, 0.33789062, -0.46289..."
2,Science,Science,Unknown,News,Meteor Showers in 2020 That Will Light Up Nigh...,"All year long, Earth passes through streams of...","['Meteors and Meteorites', 'Space and Astronom...",1057,2020-01-01 05:00:08+00:00,6,nyt://article/04bc90f0-b20b-511c-b5bb-3ce13194...,"[0.103515625, -0.47265625, 0.35546875, 0.11230...","[0.103515625, -0.47265625, 0.35546875, 0.11230..."
3,Science,Science,Unknown,Interactive Feature,Sync your calendar with the solar system,"Never miss an eclipse, a meteor shower, a rock...","['Space and Astronomy', 'Moon', 'Eclipses', 'S...",0,2020-01-01 05:00:12+00:00,2,nyt://interactive/5b58d876-9351-50af-9b41-a312...,"[-0.34375, 0.33789062, 0.22851562, 0.21386719,...","[-0.34375, 0.33789062, 0.22851562, 0.21386719,..."
4,Science,Science,Unknown,News,"Rocket Launches, Trips to Mars and More 2020 S...",A year full of highs and lows in space just en...,"['Space and Astronomy', 'Private Spaceflight',...",1156,2020-01-01 05:02:38+00:00,25,nyt://article/bd8647b3-8ec6-50aa-95cf-2b81ed12...,"[0.16210938, 0.025390625, -0.10449219, -0.5117...","[0.16210938, 0.025390625, -0.10449219, -0.5117..."


In [60]:
# Display memory usage
memory_usage = df1.memory_usage(deep=True).sum() / (1024**2)  # in megabytes
print(f"Memory Usage: {memory_usage:.2f} MB")

Memory Usage: 424.46 MB


In [85]:
# randomlly selete news/content for testing

import random
random_seed = random.randint(1, 1000)

# Randomly select 1 records from df1 with the generated random seed
random_records = df1.sample(n=1, random_state=random_seed)

# Assign the randomly selected records to df1_random
df1_random = random_records.copy()

In [86]:
df1_random.head()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,n_comments,uniqueID,np_array,embedding
12629,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492..."


In [63]:
import pandas as pd
import json

# Load the JSON data from content_vectors.json
with open('ads_vectors.json', 'r') as file:
    data = json.load(file)

# Create a DataFrame from the loaded JSON data
df2 = pd.DataFrame(data)

# Display the DataFrame
df2.head ()

,ad,ad_id,np_array,embedding
0,"MTNG Women's 69439 Trainers, (Soft Metallic Li...",2023-1875908,"[-0.21484375, 0.59375, -0.027709961, -0.090820...","[-0.21484375, 0.59375, -0.027709961, -0.090820..."
1,Fremont Die NCAA Florida State Seminoles Boat ...,2023-2093788,"[-0.15136719, 0.05834961, -0.328125, 0.328125,...","[-0.15136719, 0.05834961, -0.328125, 0.328125,..."
2,10 Label/card Holder Nickel Plated 5/8x2 1/2 W...,2023-2237808,"[-0.09472656, 0.49804688, -0.15429688, 0.15625...","[-0.09472656, 0.49804688, -0.15429688, 0.15625..."
3,Digital AM FM Portable Pocket Radio with Alarm...,2023-1190215,"[0.3203125, 0.15820312, 0.013549805, 0.4179687...","[0.3203125, 0.15820312, 0.013549805, 0.4179687..."
4,Lcolyoli Tragus Earrings 16G Surgical Steel & ...,2023-1394847,"[0.18359375, -0.041503906, 0.056884766, -0.617...","[0.18359375, -0.041503906, 0.056884766, -0.617..."


In [64]:
# Display memory usage
memory_usage = df2.memory_usage(deep=True).sum() / (1024**2)  # in megabytes
print(f"Memory Usage: {memory_usage:.2f} MB")

Memory Usage: 122.34 MB


## Compare the similarity 

In [87]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Initialize df3 before the loop
df3 = pd.DataFrame()


# Initialize a list to collect data
df3_list = []

# Loop over each record in df2['np_array']
for index2, row2 in df2.iterrows():
    ad_id = row2['ad_id']
    ad_np_array = row2['np_array']
    
    similarities = []
    
    # Loop over each record in df1['np_array']
    for index1, row1 in df1_random.iterrows():
        content_id = row1['uniqueID']
        abstract = row1['abstract']
        content_np_array = row1['np_array']
        
        # Calculate cosine similarity
        similarity = cosine_similarity([ad_np_array], [content_np_array])[0][0]
        
        similarities.append({'ad_id': ad_id, 'similarity': similarity})
    
    # Sort by similarity and select top 20
    top_20_similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)[:20]
    
    # Create ad_id_array with scores
    ad_id_array = {item['ad_id']: item['similarity'] for item in top_20_similarities}
    
    # Create a dictionary for the current row in df1
    row1_dict = row1.to_dict()
    row1_dict['ad_id_array'] = ad_id_array
    
    # Append the dictionary to the list
    df3_list.append(row1_dict)

# Concatenate all dictionaries into a DataFrame
df3 = pd.concat([pd.DataFrame([item]) for item in df3_list], ignore_index=True)

# df3 now contains the desired result


In [88]:
df3.head ()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,n_comments,uniqueID,np_array,embedding,ad_id_array
0,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-1875908': 0.06635289899548105}
1,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2093788': 0.1253709907868514}
2,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2237808': 0.01828267658429854}
3,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-1190215': 0.018612271726144113}
4,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-1394847': -0.002057430250027587}


## Re-ranking by the score, from higher to lower

In [90]:

# Extract the values from the dictionaries in the ad_id_array column
df3['score'] = df3['ad_id_array'].apply(lambda x: list(x.values())[0])

# Sort the DataFrame based on the score column
df3_sorted = df3.sort_values(by='score', ascending=False).reset_index(drop=True)

# Display the updated DataFrame
df3_sorted.head (5)


,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,n_comments,uniqueID,np_array,embedding,ad_id_array,score
0,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2016477': 0.2909663557762484},0.290966
1,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2082549': 0.27672798734875304},0.276728
2,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2046523': 0.2718515880072647},0.271852
3,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2037634': 0.26991900400949603},0.269919
4,Sports,Sports,Pro Football,News,Jackson-Mahomes III: ‘The Whole World Is Watch...,"They’re under 26, have won M.V.P.s and they ca...","['Football', 'Kansas City Chiefs', 'Baltimore ...",1050,2020-09-28 07:00:08+00:00,26,nyt://article/6af4e6a8-009e-5034-9630-f7f0c89e...,"[-0.37890625, -0.546875, 0.11425781, -0.104492...","[-0.37890625, -0.546875, 0.11425781, -0.104492...",{'2023-2055505': 0.26852532665716244},0.268525


### Retrieve the ad description by ad_id in the top 1 ranking

In [ ]:
# check the top 1 ads's description
desired_ad_id = '2023-2082549'

# Use boolean indexing to filter rows where ad_id is equal to the desired value
filtered_rows = df2[df2['ad_id'] == desired_ad_id]

# If there is exactly one row with the specified ad_id, get the value of 'ad'
if len(filtered_rows) == 1:
    desired_ad_value = filtered_rows['ad'].values[0]
    print("The value of 'ad' for ad_id '{}' is: {}".format(desired_ad_id, desired_ad_value))
else:
    print("No unique value found for ad_id '{}' or multiple rows found.".format(desired_ad_id))


The value of 'ad' for ad_id '2023-2082549' is: THE NORTHWEST COMPANY NFL San Diego Chargers Flair Hair Adjustable Visor, Navy


### Retrieve the news description; you can justify it with your human sense

In [104]:
import json

file_path = 'output_Ads_to_content_similarity.json'

with open(file_path, 'r') as file:
    # Read only the first line
    line = file.readline()
    
    # Check if there is any data in the line
    if line:
        data = json.loads(line)
        if 'abstract' in data:
            print('Abstract:', data['abstract'])
    else:
        print('No data found in the file.')




Abstract: They’re under 26, have won M.V.P.s and they can seemingly do anything. On Monday night, the N.F.L. gets a boost from the latest installment of its best current rivalry.


In [93]:

df3_sorted.to_json('output_Ads_to_content_similarity.json', orient='records', lines=True)


### Summary, from the test result, it will show you whihc are the most relevance ads for the contents